This note book has 5 main steps:


1. Load the checkpoint
2. Score the file created by mBART by translating `english` to `tamil`
3. Score the file created by IndicTrans by translating `english` to `tamil`
4. Combine the logits from step 2, step 3, with the original sentence and the translated senences. This is also saved for further analysis
5. Apply `softmax` on the logits from step 2 and 3, resulting in `selecting` the better translations

### Step 1

In [1]:
%cd indic-bert

/w266/ucb/w266_project/TranQuality/finetuning_indicBERT/indic-bert


In [2]:
import glob
import os

In [3]:
from fine_tune.modules import get_modules
from fine_tune.data import load_dataset
from fine_tune.data.examples import *
from fine_tune.modules import base

In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

In [5]:
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [6]:
# Softmax calculation
def _softmax(x1, x2, tog=0):
    a = np.exp([x1,x2])/np.sum(np.exp([x1,x2]))
    if tog == 1:
        return a[0]
    else:
        return a[1]

def apply_softmax(df, tog):
    return df.apply(lambda row: _softmax(row[0], row[1], tog), axis=1)

In [7]:
# Define hyperparameters for testing
hparams = {
    'dataset': 'amrita-paraphrase-exact', 
    'model': 'ai4bharat/indic-bert', 
    'config_name': '', 
    'tokenizer_name': '', 
    'max_seq_length': 256, 
    'iglue_dir': '../indic-glue', 
    'overwrite_cache': True, 
    'cache_dir': '../ib-cache', 
    'fp16': False, 
    'fp16_opt_level': 'O1', 
    'n_gpu': 1, 
    'n_tpu_cores': 0, 
    'max_grad_norm': 1.0, 
    'do_train': True, 
    'do_predict': True, 
    'gradient_accumulation_steps': 1,
    'seed': 8, 
    'learning_rate': 2e-05, 
    'weight_decay': 0.0, 
    'adam_epsilon': 1e-08, 
    'warmup_steps': 0, 
    'num_train_epochs': 1, 
    'train_batch_size': 16, 
    'eval_batch_size': 32, 
    'labels': '', 
    'model_name_or_path': 'ai4bharat/indic-bert', 
    'data_dir': '../indic-glue/amrita-paraphrase-exact', 
    'mode': 'sequence-classification', 
    'output_mode': 'regression', #'classification', 
    'example_type': 'text', 
    'num_labels': 2
}

In [8]:
# Set up parameters for Tamil
hparams['out_dir'] = '../outputs/3/'
hparams['lang'] = 'ta' 
hparams['train_lang'] = 'ta'
hparams['dev_lang'] = 'ta'
hparams['test_lang'] = 'ta'
hparams['output_dir'] = '{}amrita-paraphrase-exact/train-{}/model-ai4bharat-indic-bert'.format(hparams['out_dir'],hparams['lang'] ) 

In [9]:
config_name = hparams['config_name'] or hparams['model_name_or_path']
args = {'num_labels': hparams['num_labels']}
config = AutoConfig.from_pretrained(
    config_name,
    **args,
    cache_dir=hparams['cache_dir']
)

## Step 2 - Score mBART Tamil

In [10]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [11]:
!cp ../indic-glue/amrita-paraphrase-exact/ta/ta-bart-test.csv ../indic-glue/amrita-paraphrase-exact/ta/ta-test.csv 

In [12]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ta/ta-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [13]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

In [14]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [15]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [16]:
trainer.test(trained_model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Logits written to :../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert/test_results_ta.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.594,
 'avg_test_loss': 1.5797350406646729,
 'val_loss': 1.5797350406646729}
--------------------------------------------------------------------------------
{'avg_test_loss': 1.5797350406646729, 'val_loss': 1.5797350406646729, 'acc': 0.594}



## Load the scores for mBART translations.

In [17]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert/test_results_ta.csv'

In [18]:
mbart = pd.read_csv(hparams['results_file'], header=None)

In [19]:
mbart.shape

(1000, 4)

In [20]:
mbart.head(5)

,0,1,2,3
0,-0.129751,0.201876,1.0,1.0
1,1.858723,-2.035915,0.0,1.0
2,-0.164003,0.236660,1.0,1.0
3,-0.170018,0.243252,1.0,1.0
4,-0.093489,0.150331,1.0,1.0


## Step 3 - Score IndicTrans Tamil

In [21]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [22]:
!cp ../indic-glue/amrita-paraphrase-exact/ta/ta-indictrans-test.csv ../indic-glue/amrita-paraphrase-exact/ta/ta-test.csv 


In [23]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ta/ta-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [24]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

In [25]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [26]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [27]:
trainer.test(trained_model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Logits written to :../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert/test_results_ta.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.346,
 'avg_test_loss': 0.6001479029655457,
 'val_loss': 0.6001479029655457}
--------------------------------------------------------------------------------
{'avg_test_loss': 0.6001479029655457, 'val_loss': 0.6001479029655457, 'acc': 0.346}



## Load the scores for IndicTrans translations.

In [28]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/3/amrita-paraphrase-exact/train-ta/model-ai4bharat-indic-bert/test_results_ta.csv'

In [29]:
indictrans = pd.read_csv(hparams['results_file'], header=None)

In [30]:
indictrans.shape

(1000, 4)

In [31]:
indictrans.head(5)

,0,1,2,3
0,-0.138827,0.210006,1.0,0.0
1,1.857611,-2.018302,0.0,0.0
2,1.786160,-1.915916,0.0,0.0
3,-0.167744,0.239971,1.0,0.0
4,-0.104022,0.164458,1.0,0.0


## Step 4 -  Combine logits, original sentence, and translations

In [32]:
indic_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ta/ta-indictrans-test.csv', header=None)

In [33]:
indic_orig.head()

,0,1,2
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,NP
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",NP
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...,NP
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...,NP
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",NP


In [34]:
bart_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ta/ta-bart-test.csv', header=None)

In [35]:
bart_orig.head()

,0,1,2
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",P
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,P
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,P
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,P
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...",P


In [36]:
combined = bart_orig[[0, 1]]

In [37]:
combined.columns =['Original_Sentence', 'mBART_Translated', ]

In [38]:
combined['IndicTrans_Translated'] = indic_orig[[1]]

In [39]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்..."
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,..."


### Step 5 - `apply_softmax`

In [41]:
combined['mBART_NP_prob'] = mbart[[0]]
combined['mBART_P_prob'] = mbart[[1]]
combined['model_pred_mbart'] = ['NP' if x==0 else 'P' for x in mbart[2]]
combined['mbart_P'] = apply_softmax(mbart, 0)
combined['mbart_NP'] = apply_softmax(mbart, 1)

In [42]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,-0.129751,0.201876,P,0.582155,0.417845
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",1.858723,-2.035915,NP,0.019945,0.980055
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...,-0.164003,0.236660,P,0.598847,0.401153
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...,-0.170018,0.243252,P,0.601872,0.398128
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",-0.093489,0.150331,P,0.560655,0.439345


In [43]:
combined['IndicT_NP_prob'] = indictrans[[0]]
combined['IndicT_P_prob'] = indictrans[[1]]
combined['model_pred_IndicT'] = ['NP' if x==0 else 'P' for x in indictrans[2]]
combined['indicT_P'] = apply_softmax(indictrans, 0)
combined['indicT_NP'] = apply_softmax(indictrans, 1)

In [44]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,-0.129751,0.201876,P,0.582155,0.417845,-0.138827,0.210006,P,0.586335,0.413665
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",1.858723,-2.035915,NP,0.019945,0.980055,1.857611,-2.018302,NP,0.020314,0.979686
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...,-0.164003,0.236660,P,0.598847,0.401153,1.786160,-1.915916,NP,0.024078,0.975922
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...,-0.170018,0.243252,P,0.601872,0.398128,-0.167744,0.239971,P,0.600540,0.399460
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",-0.093489,0.150331,P,0.560655,0.439345,-0.104022,0.164458,P,0.566720,0.433280


In [45]:
def select_trasnlation(row):
    if row['indicT_P']>=row['mbart_P']:
        return row['IndicTrans_Translated']
    else:
        return row['mBART_Translated']

combined['selected_translation'] = combined.apply(lambda row : select_trasnlation(row), axis=1) 

In [46]:
combined['mBART_avg'] = combined.iloc[:,3:5].mean(axis=1)

In [47]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP,selected_translation,mBART_avg
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,-0.129751,0.201876,P,0.582155,0.417845,-0.138827,0.210006,P,0.586335,0.413665,கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,0.036063
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",1.858723,-2.035915,NP,0.019945,0.980055,1.857611,-2.018302,NP,0.020314,0.979686,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",-0.088596
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...,-0.164003,0.236660,P,0.598847,0.401153,1.786160,-1.915916,NP,0.024078,0.975922,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,0.036329
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...,-0.170018,0.243252,P,0.601872,0.398128,-0.167744,0.239971,P,0.600540,0.399460,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,0.036617
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",-0.093489,0.150331,P,0.560655,0.439345,-0.104022,0.164458,P,0.566720,0.433280,"""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",0.028421


In [48]:
combined['IT_avg'] = combined.iloc[:,5:7].mean(axis=1)

In [49]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP,selected_translation,mBART_avg,IT_avg
0,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு...",கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,-0.129751,0.201876,P,0.582155,0.417845,-0.138827,0.210006,P,0.586335,0.413665,கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...,0.036063,0.582155
1,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",1.858723,-2.035915,NP,0.019945,0.980055,1.857611,-2.018302,NP,0.020314,0.979686,"""பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன்...",-0.088596,0.019945
2,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...,-0.164003,0.236660,P,0.598847,0.401153,1.786160,-1.915916,NP,0.024078,0.975922,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...,0.036329,0.598847
3,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...,-0.170018,0.243252,P,0.601872,0.398128,-0.167744,0.239971,P,0.600540,0.399460,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...,0.036617,0.601872
4,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப...","""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",-0.093489,0.150331,P,0.560655,0.439345,-0.104022,0.164458,P,0.566720,0.433280,"""இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல்,...",0.028421,0.560655


In [50]:
combined.to_csv('../combined-ta.csv', sep='\t', encoding='utf-8')

In [51]:
np.savetxt('../original_sentence-ta.txt', combined['Original_Sentence'], fmt='%s')
np.savetxt('../mBART_translated-ta.txt', combined['mBART_Translated'], fmt='%s')
np.savetxt('../indictrans_translated-ta.txt', combined['IndicTrans_Translated'], fmt='%s')
np.savetxt('../selected_translation-ta.txt', combined['selected_translation'], fmt='%s')

In [52]:
!cp ../original_sentence-ta.txt ../../bleu_test/testing/indicTrans/
!cp ../mBART_translated-ta.txt  ../../bleu_test/testing/indicTrans/
!cp ../indictrans_translated-ta.txt  ../../bleu_test/testing/indicTrans/
!cp ../selected_translation-ta.txt  ../../bleu_test/testing/indicTrans/